Q1) Rabbit Problem

In [11]:

class State:
    def __init__(self, state):
        self.state = state

    def __eq__(self, otherState):
        return self.state == otherState.state

    def __hash__(self):
        return hash(tuple(self.state))

    def __str__(self):
        return '['+' '.join(self.state)+']'

    def goalTest(self):
        return self.state == ['L', 'L', 'L', '_', 'R', 'R', 'R']

    def moveGen(self):
        children = []
        idx = self.state.index('_')
        steps = [-1, -2, 1, 2]
        for d in steps:
            new_idx = idx + d
            if 0 <= new_idx < 7:
                if abs(d) == 1 or (abs(d) == 2 and self.state[idx + d // 2] != '_'):
                    new_state = self.state.copy()
                    new_state[idx], new_state[new_idx] = new_state[new_idx], new_state[idx]
                    children.append(State(new_state))
        return children


BFS AND DFS Approach for both questions

In [2]:
def reconstructPath(goal_node_pair, CLOSED):
    parent_map = {}
    for node, parent in CLOSED:
        parent_map[node] = parent
    
    path = []
    goal_node, parent = goal_node_pair
    path.append(goal_node)
    while parent is not None:
        path.append(parent)
        parent = parent_map[parent]
    
    return path


def removeSeen(children, OPEN, CLOSED):
    open_nodes = [node for node, parent in OPEN]
    closed_nodes = [node for node, parent in CLOSED]
    new_nodes = [c for c in children if c not in open_nodes and c not in closed_nodes]
    return new_nodes
    

def bfs(start):
    OPEN = [(start, None)]
    CLOSED = []
    while OPEN:
        node_pair = OPEN.pop(0)
        N, parent = node_pair
        
        if N.goalTest():
            print("Goal is found")
            path = reconstructPath(node_pair, CLOSED)
            path.reverse()
            for node in path:
                print(node, " -> ",end="")
            return
        else:
            CLOSED.append(node_pair)
            children = N.moveGen()
            new_nodes = removeSeen(children, OPEN, CLOSED)
            new_pairs = [(node, N) for node in new_nodes]
            OPEN = OPEN + new_pairs
    return []

#DFS
def dfs(start):
    OPEN = [(start, None)]
    CLOSED = []
    while OPEN:
        node_pair = OPEN.pop(0)
        N, parent = node_pair
        
        if N.goalTest():
            print("Goal is found")
            path = reconstructPath(node_pair, CLOSED)
            path.reverse()
            for node in path:
                print(node, " -> ",end="")
            return
        else:
            CLOSED.append(node_pair)
            children = N.moveGen()
            new_nodes = removeSeen(children, OPEN, CLOSED)
            new_pairs = [(node, N) for node in new_nodes]
            OPEN = new_pairs + OPEN #LIFO
    return []

In [12]:
start_state = State(['R', 'R', 'R', '_', 'L', 'L', 'L'])

print("BFS:")
bfs(start_state)

print("\nDFS:")
dfs(start_state)


BFS:
Goal is found
[R R R _ L L L]  -> [R R _ R L L L]  -> [R R L R _ L L]  -> [R R L R L _ L]  -> [R R L _ L R L]  -> [R _ L R L R L]  -> [_ R L R L R L]  -> [L R _ R L R L]  -> [L R L R _ R L]  -> [L R L R L R _]  -> [L R L R L _ R]  -> [L R L _ L R R]  -> [L _ L R L R R]  -> [L L _ R L R R]  -> [L L L R _ R R]  -> [L L L _ R R R]  -> 
DFS:
Goal is found
[R R R _ L L L]  -> [R R _ R L L L]  -> [R R L R _ L L]  -> [R R L _ R L L]  -> [R R _ L R L L]  -> [R _ R L R L L]  -> [R L R _ R L L]  -> [R L _ R R L L]  -> [_ L R R R L L]  -> [L _ R R R L L]  -> [L R _ R R L L]  -> [L R R R _ L L]  -> [L R R R L _ L]  -> [L R R _ L R L]  -> [L R _ R L R L]  -> [_ R L R L R L]  -> [R _ L R L R L]  -> [R L _ R L R L]  -> [R L R _ L R L]  -> [R _ R L L R L]  -> [R R _ L L R L]  -> [R R L L _ R L]  -> [R R L L L R _]  -> [R R L L L _ R]  -> [R R L L _ L R]  -> [R R _ L L L R]  -> [R _ R L L L R]  -> [R L R _ L L R]  -> [R L _ R L L R]  -> [R _ L R L L R]  -> [_ R L R L L R]  -> [L R _ R L L R]  -> [

Q)2  Bridge Problem

In [28]:
class State:
    def __init__(self, Amogh, Ameya, Grandmother, Grandfather, time, umbrella_side):
        self.positions = {
            "Amogh": Amogh,
            "Ameya": Ameya,
            "Grandmother": Grandmother,
            "Grandfather": Grandfather
        }
        self.time = time
        self.umbrella_side = umbrella_side

    def __eq__(self, other):
        return self.positions == other.positions and self.time == other.time

    def __hash__(self):
        return hash((tuple(sorted(self.positions.items())), self.time,self.umbrella_side))

    def __str__(self):
            pos_list = [f"{person} : {side}" for person, side in self.positions.items()]
            return f"[{pos_list}, Time: {self.time}, Umbrella: {self.umbrella_side}]\n"

    def goalTest(self):
        return all(side == "R" for side in self.positions.values()) and self.time <= 60

    def moveGen(self):
        children = []
        times = {
            "Amogh": 5,
            "Ameya": 10,
            "Grandmother": 20,
            "Grandfather": 25
        }

        candidates = [p for p, side in self.positions.items() if side == self.umbrella_side]

        if self.umbrella_side == "L":
            for i in range(len(candidates)):
                for j in range(i+1, len(candidates)):
                    p1, p2 = candidates[i], candidates[j]
                    new_positions = self.positions.copy()
                    new_positions[p1] = "R"
                    new_positions[p2] = "R"
                    new_time = self.time+ max(times[p1], times[p2])
                    if new_time <= 60:
                        child_obj = State(**new_positions, time=new_time,umbrella_side="R")
                        children.append(child_obj)
        else:
            for p in candidates:
                new_positions = self.positions.copy()
                new_positions[p] = "L"
                new_time = self.time + times[p]
                if new_time <= 60:
                    child = State(**new_positions, time=new_time,umbrella_side="L")
                    children.append(child)

        return children

In [29]:
start = State(
    Amogh="L",
    Ameya="L",
    Grandmother="L",
    Grandfather="L",
    time=0,
    umbrella_side="L"
)

print("\nBFS")
bfs(start)

print("\nDFS")
dfs(start)



BFS
Goal is found
[['Amogh : L', 'Ameya : L', 'Grandmother : L', 'Grandfather : L'], Time: 0, Umbrella: L]
  -> [['Amogh : R', 'Ameya : R', 'Grandmother : L', 'Grandfather : L'], Time: 10, Umbrella: R]
  -> [['Amogh : L', 'Ameya : R', 'Grandmother : L', 'Grandfather : L'], Time: 15, Umbrella: L]
  -> [['Amogh : L', 'Ameya : R', 'Grandmother : R', 'Grandfather : R'], Time: 40, Umbrella: R]
  -> [['Amogh : L', 'Ameya : L', 'Grandmother : R', 'Grandfather : R'], Time: 50, Umbrella: L]
  -> [['Amogh : R', 'Ameya : R', 'Grandmother : R', 'Grandfather : R'], Time: 60, Umbrella: R]
  -> 
DFS
Goal is found
[['Amogh : L', 'Ameya : L', 'Grandmother : L', 'Grandfather : L'], Time: 0, Umbrella: L]
  -> [['Amogh : R', 'Ameya : R', 'Grandmother : L', 'Grandfather : L'], Time: 10, Umbrella: R]
  -> [['Amogh : L', 'Ameya : R', 'Grandmother : L', 'Grandfather : L'], Time: 15, Umbrella: L]
  -> [['Amogh : L', 'Ameya : R', 'Grandmother : R', 'Grandfather : R'], Time: 40, Umbrella: R]
  -> [['Amogh : L',